# Station

Here, the following topics are going to be covered:

- What is a station
- How to create it, and work with it
- Snapshot of a station
- Configuring station using a YAML configuration file

### Useful imports

In [1]:
from pprint import pprint  # for pretty-printing python variables like 'dict'

import qcodes
from qcodes import Parameter, Station
from qcodes.tests.instrument_mocks import DummyInstrument

## What is a Station

Experimental setups are large, comprising of many instruments; and instruments tend to be quite complex in that they comprise many parameters and other stateful parts. It deems useful to have a bucket where all of them can be conveniently stored, and accessed.

Here is where the concept of station comes into play. Instruments, parameters, and other "components" can be added to a station. As a result, the user gets an station instance that can be referred to in order to access those "components".

Moreover, stations are very helpful when capturing the state of the experimental setup, known as snapshot. Refer to the respective section about station snapshot below.

Last but not least, station can be configured in a text file which simplifies initialization of the instruments. Read more of this below.

## How to create Station and work with it

For further sections we will need a dummy parameter, and a dummy instrument.

In [2]:
# A dummy self-standing parameter
p = Parameter('p', label='Parameter P', unit='kg', set_cmd=None, get_cmd=None)
p.set(123)

In [3]:
# A dummy instrument with three parameters
instr = DummyInstrument('instr', gates=['input', 'output', 'gain'])
instr.gain(42)

### Creating a Station

In [4]:
station = Station()

station.add_component(p)
station.add_component(instr)

'instr'

In [5]:
# Now station contains both `p` and `instr`
station.components

{'p': <qcodes.instrument.parameter.Parameter: p at 4817387984>,
 'instr': <DummyInstrument: instr>}

Note that it is also possible to add components to a station via arguments of its constructor, like this:

In [6]:
station = Station(p, instr)

### Access Station components

Now that the components have beed added to the station, it is possible to access them as attributes of the station (using the "dot" notation). With this feature, users can use tab-completion to find the instrument in the station they'd like to access.

In [7]:
# Let's confirm that station's `p` is 
# actually the `p` parameter defined above
assert station.p is p

### Removing component from Station

Removing components from the station should be done with `remove_component` method - just pass it a name of the component you'd like to remove:

In [8]:
station.remove_component('p')

<qcodes.instrument.parameter.Parameter: p at 4817387984>

In [9]:
# Now station contains only `instr`
station.components

{'instr': <DummyInstrument: instr>}

### Default Station

The `Station` class is designed in such a way that it always contains a reference to a `default` station object (the `Station.default` attribute). The constructor of the station object has a `default` keyword argument that allows to specify whether the resulting instance shall be stored as a default station, or not.

This feature is a convenience. Other objects which consume an instance of `Station` as an argument (for example, `Measurement`) can now implement a logic to resort to `Station.default` in case an `Station` instance was not explicitly given to them.

## Snapshot of a Station

The station has a `snapshot` method that allows to create a collective, single snapshot of all the instruments, parameters, and submodules that have been added to it. It would be very time-consuming for the user to manually go through every instrument and parameter, and collect the snapshot data.

For example, the `Measurement` object accepts a station argument exactly for the purpose of storing a snapshot of the whole experimental setup next to the measured data.

Read more about snapshots in general, how to work with them, station's snapshot in particular, and more -- in ["Working with snapshots" example notebook](DataSet/Working with snapshots.ipynb) ([nbviewer.jupyter.org link](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/DataSet/Working with snapshots.ipynb)).

## Configuring Station using YAML configuration file

The initialization part of the code where one instantiates instruments, set's up proper initial values for parameters, etc. can be quite long and tedious to maintain. For example, when a certain instrument is no longer needed, usually users just comment out the lines of initialization script which are related to the said instrument, and re-run the initialization script. Sharing initialization scripts is also difficult because each user may have a different expecation on the format it.

These (and more) concerns are to be solved by YAML configuration of the `Station` (formerly known to some users under the name `StationConfigurator`).

The YAML configuration file allows to statically and uniformly specify settings of all the instruments (and their parameters) that the measurement setup (the "physical" station) consists of, and load them with those settings on demand. The `Station` object implements convenient methods for this.

The YAML configuration, if used, is stored in the station as a component with name `config`, and is thus included in the snapshot of the whole station.

Note that one is not obliged to use the YAML configuration for setting up one's `Station` (for example, as at the top of this notebook).

Below the following is discussed:

- The structure of the YAML configuration file
- `Station`s methods realted to working with the YAML configuration
- Entries in QCoDeS config that are related to `Station`

### Example of YAML Station configuration

Below is an example YAML station configuration file. All the fields are explained in the inline comments. Read it through.

When exploring the YAML file capabilities, please note the difference between `parameters` section and `add_parameters` section. In the example file below, for the `QDac` instrument, the `Bx` parameter is going to be a new, additional parameter. This new `Bx` parameter will have its `limits`, `scale`, etc. __different__ from its "source" parameter `ch02.v` that it controls. Specifically this means that when setting `Bx` to `2.0`:

1. the value of `2.0` is being validated against the limits of `Bx` (`0.0, 3.0`),
2. then the raw ("scaled") value of `130.468` (`= 2.0 * 65.234`) is passed to the `ch02.v` parameter,
3. then that value of `130.468` is validated against the limits of `ch02.v` (`0.0, 1.5e+3`),
4. then the raw ("scaled") value of `1.30468` (`= 130.468 * 0.01`) is finally passed to the physical instrument.

Please also note that when trying in numbers in exponential form, it is required to provide `+` and `-` signs after `e`, for example, `7.8334e+5` and `2.5e-23`. Refer to YAML file format specification for more information.

```yaml
# Example YAML Station configuration file
#
# This file gets snapshotted and can be read back from the JSON 
# snapshot for every experiment run.
#
# All fields are optional unless explicitly mentioned otherwise.
#
# As in all YAML files a one-line notation can also be used
# instead of nesting notation.
#
# The file starts with a list of loadable instruments instances,
# i.e. there can be two entries for two instruments of the same
# type if you want to specify two different use cases 
# e.g. "dmm1-readout" and "dmm1-calibration".
#
instruments:
  # Each instrument is specified by its name.
  # This name is what is looked up by the `load_instrument`
  # method of `Station`.
  # Simulated instruments can also be specified here, just put
  # the path to the similation .yaml file as the value of the
  # "init"->"visalib" field (see below for an example of the
  # "init" section as well as an example of specifying 
  # a simulated instrument).
  qdac:
    # Full import path to the python module that contains
    # the driver class of the instrument.
    # Required field.
    driver: qcodes.instrument_drivers.QDev.QDac_channels
    # The name of the class of the instrument driver.
    # Required field.
    type: QDac
    # Visa address of the instrument.
    # Note that this field can also be specified in the
    # "init" section (see below) but the address specified 
    # here will overrule the address from the "init" section.
    # Essentially, specifying address here allows avoiding
    # the "init" section completely when address is the only
    # neccesary argument that the instrument driver needs.
    # For obvious reasons, this field is required for VISA
    # instruments.
    address: ASRL4::INSTR
    # If an instrument with this name is already instantiated,
    # and this field is true, then the existing instrument 
    # instance will be closed before instantiating this new one.
    # If this field is false, or left out, closing will not
    # happen.
    enable_forced_reconnect: true
    #
    # The "init" section specifies constant arguments that are
    # to be passed to the __init__ function of the instrument.
    # Note that it is the instrument's driver class that defines
    # the allowed arguments, for example, here "update_currents"
    # is an argument that is specific to "QDac" driver.
    init:
      terminator: \n
      update_currents: false
    #
    # Setting up properties of parameters that already exist on
    # the instrument.
    parameters:
      # Each parameter is specified by its name from the
      # instrument driver class.
      # Note that "dot: notation can be used to specify 
      # parameters in (sub)channels and submodules.
      ch01.v:
        # If an alias is specified, the paramater becomes  
        # accessible under another name, so that you can write
        # `qdac.cutter_gate(0.2)` instead of `qdac.ch01.v(0.2)`.
        # Note that the parameter instance does not get copied,
        # so that `(qdac.ch01.v is qdac.cutter_gate) == True`.
        alias: cutter_gate
        # Set new label.
        label: Cutter Gate Voltage
        # Set new unit.
        unit: mV
        # Set new scale.
        scale: 0.001
        # Set new post_delay.
        post_delay: 0
        # Set new inter_delay.
        inter_delay: 0.01
        # Set new step.
        step: 1e-4
        # If this field is given, and contains two 
        # comma-separated numbers like here, then the parameter
        # gets a new `Numbers` validator with these values as
        # lower and upper limits, respectively (in this case, it
        # is `Numbers(-0.1, 0.1)`).
        limits: -0.1,0.1
        # Set the parameter to this given initial value upon
        # instrument initialization.
        # Note that if the current value on the physical
        # instrument is different, the parameter will be ramped
        # with the delays and step specified in this file.
        initial_value: 0.01
        # In case this values equals to true, upon loading this
        # instrument from this configuration this parameter will
        # be appended to the list of parameters that are 
        # displayed in QCoDeS `Monitor`.
        monitor: true
      # As in all YAML files a one-line notation can also be 
      # used, here is an example.
      ch02.v: {scale: 0.01, limits: '0.0,1.5e+3', label: my label}
      ch04.v: {alias: Q1lplg1, monitor: true}
    #
    # This section allows to add new parameters to the
    # instrument instance which are based on existing parameters
    # of the instrument. This functionality is based on the use
    # of the `DelegateParameter` class.
    add_parameters:
      # For example, here we define a parameter that represents
      # magnetic field control. Setting and getting this 
      # parameter will actually set/get a specific DAC channel.
      # So this new magnetic field parameter is playing a role
      # of a convenient proxy - it is much more convenient to 
      # perform a measurement where "Bx" is changed in tesla as
      # opposed to where some channel of some DAC is changed in
      # volts and one has to clutter the measurement code with
      # the mess of conversion factors and more.
      # Every new parameter definition starts with a name of
      # the new parameter.
      Bx:
        # This field specifies the parameter which "getter" and 
        # "setter" will be used when calling `get`/`set` on this
        # new parameter.
        # Required field.
        source: ch02.v
        # Set the label. Otherwise, the one of the source parameter
        # will be used.
        label: Magnetic Field X-Component
        # Set the unit. Otherwise, the one of the source parameter
        # will be used.
        unit: T
        # Other fields have the same purpose and behavior as for
        # the entries in the `add_parameter` section.
        scale: 65.243
        inter_delay: 0.001
        post_delay: 0.05
        step: 0.001
        limits: 0.0,3.0
        initial_value: 0.0
        # For the sake of example, we decided not to monitor this
        # parameter in QCoDeS `Monitor`.
        #monitor: true
  #
  # More example instruments, just for the sake of example.
  # Note that configuring simulated instruments also works,
  # see the use of 'visalib' argument field below
  dmm1:
    driver: qcodes.instrument_drivers.agilent.Agilent_34400A
    type: Agilent_34400A
    enable_forced_reconnect: true
    address: GPIB::1::65535::INSTR
    init:
      visalib: 'Agilent_34400A.yaml@sim'
    parameters:
      volt: {monitor: true}
  mock_dac:
    driver: qcodes.tests.instrument_mocks
    type: DummyInstrument
    enable_forced_reconnect: true
    init:
      # To pass an list of items use {}.
      gates: {"ch1", "ch2"}
    add_parameters:
      Bx: {source: ch1, label: Bx, unit: T,
           scale: 28, limits: "-1,1", monitor: true}
  mock_dac2:
    driver: qcodes.tests.instrument_mocks
    type: DummyInstrument
    enable_forced_reconnect: true
```

### QCoDeS config entries related to Station

QCoDeS config contains entries that are related to the `Station` and its YAML configuration. Refer to [the description of the 'station' section in QCoDeS config](http://qcodes.github.io/Qcodes/user/configuration.html?highlight=station#default-config) for more specific information.

### Using Station with YAML configuration file

This section is going to briefly describe the usage of `Station` with the YAML configuration file. For details, refer to the docstrings of the methods of the `Station` class.

A `Station` with a YAML configuration file can be created by passing the file name (or file name with absolute path) to `Station`s constructor. File name and location resolution also takes into account related entries in the `'station'` section of the QCoDeS config, refer to their documentation for more information.

```python
station = Station(config_file='qutech_station_25.yaml')
```

Alternatively, `load_config_file` method can be called on an already instantiated station to load the config file.

```python
station = Station()
stataion.load_config_file=r'Q:\\station_data\\qutech_station_25.yaml')
```

In case the configuration is already available as a YAML string, then that configuration can be loaded using `Station`'s `load_config` method, refer to it's docstring and signature for more information.

Once the YAML configuration is loaded, the `load_instrument` method of the `Station` can be used to instantiate a particular instrument that is described in the YAML configuration. Calling this method not only will return an instance of the linstantiated instrument, but will also add it to the station object.

For example, to instantiate the `qdac` instrument from the YAML configuration example from the section above, just execute the following:
```python
loaded_qdac = station.load_instrument('qdac')
```

Note the `load_instrument`'s `revive_instance` argument, as well as `enable_force_reconnect` setting from the YAML configuration - these define what to do in case an instrument with the given name has already been instantiated in this python session.

There is a more convenient way to load the instruments. Upon load of the YAML configuration, convenient `load_<instrument name>` methods are being generated on the `Station` object. Users can make use of tab-completion in their development environments to list what instruments can be loads by the station object from the loaded YAML configuration. For example, loading the QDac above can also be done like this:
```python
conveniently_loaded_qdac = station.load_qdac()
```

Note that instruments are instantiated only when `load_*` methods are called. This means that loading a YAML configuration does NOT automatically instantiate anything.

For the instruments loaded with the `load_*` methods, it is recommended to use `Station`'s `close_and_remove_instrument` method for closing and removing those from the station.